# Animations

Movies of several variables through the experiments

In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
import cosima_cookbook as cc
import dask.distributed as dsk
import glob
import matplotlib.gridspec as gs
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import scipy.stats as st
import xarray as xr
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from palettable.colorbrewer.diverging import RdBu_11_r as rdbu_cm
from joblib import Parallel, delayed

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

figdir = '/home/561/jn8053/g_e14/figures-tmp/'

clnt = dsk.Client()
clnt

Client Scheduler: tcp://127.0.0.1:46229 Dashboard: /proxy/8787/status,Cluster Workers: 6 Cores: 24 Memory: 250.00 GiB


In [3]:
# Font size
plt.rcParams['font.size'] = 16
# Axes
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
# Saving parameters
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['savefig.bbox'] = 'tight'

In [22]:
def figure_along_slope():
    fig = plt.figure(figsize = (15, 10))
    gd = gs.GridSpec(2, 2, figure = fig, width_ratios = [1, .03])
    axs = [fig.add_subplot(gd[0,0]), fig.add_subplot(gd[1,0]), fig.add_subplot(gd[:,1])]
    for ax in axs[:-1]:
        ax.set_ylim(-80, -60);
        ax.set_xticks(np.arange(-250, 100, 50))
        ax.set_yticks(np.arange(-80, -55, 5))
    axs[0].set_title('UP')
    axs[1].set_title('DOWN')
    
    return fig, axs

In [11]:
session = cc.database.create_session()

In [12]:
mask_file = np.load('/g/data/e14/hd4873/runs/parcels/input/proj01/Antarctic_slope_contour_ht_1000m.npz')
shelf_mask = mask_file['contour_masked_above']
xt_ocean = mask_file['xt_ocean']
yt_ocean = mask_file['yt_ocean']
shelf_mask[shelf_mask != 0] = np.nan
shelf_mask[shelf_mask == 0] = 1
shelf_mask = xr.DataArray(shelf_mask, dims = ['yt_ocean', 'xt_ocean'], coords = {'yt_ocean':yt_ocean, 'xt_ocean':xt_ocean})

In [13]:
ht = cc.querying.getvar('01deg_jra55v13_ryf9091', 'ht', session, n = 1)

# Daily, first year

## SST

In [56]:
sst = {}
sst['ryf'] = cc.querying.getvar('01deg_jra55v13_ryf9091', 'surface_temp', session, frequency = '1 daily', start_time = '2150-01-01', end_time = '2150-12-31').sel(time = slice('2150-01-01', '2150-12-31')).sel(yt_ocean = slice(None, -50))
sst['eup'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_up10', 'surface_temp', session, frequency = '1 daily', start_time = '2150-01-01', end_time = '2150-12-31').sel(time = slice('2150-01-01', '2150-12-31')).sel(yt_ocean = slice(None, -50))
sst['edo'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_down10', 'surface_temp', session, frequency = '1 daily', start_time = '2150-01-01', end_time = '2150-12-31').sel(time = slice('2150-01-01', '2150-12-31')).sel(yt_ocean = slice(None, -50))

In [60]:
for k in ['ryf', 'eup', 'edo']:
    sst[k] = sst[k]*shelf_mask

In [67]:
def figures_parallel(i):
    fig, axs = figure_along_slope()
    
    axs[0].text(0.98, 0.08, str(sst['ryf'].isel(time = i)['time.year'].values)+'-'+str(sst['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[0].transAxes);
    psi = axs[0].contourf(sst['ryf']['xt_ocean'], sst['ryf']['yt_ocean'], sst['eup'].isel(time = i)-sst['ryf'].isel(time = i), 
                          levels = np.arange(-.2, .21, .01), cmap = 'RdBu_r', extend = 'both')
    axs[0].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    axs[1].text(0.98, 0.08, str(sst['ryf'].isel(time = i)['time.year'].values)+'-'+str(sst['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[1].transAxes);
    psi = axs[1].contourf(sst['ryf']['xt_ocean'], sst['ryf']['yt_ocean'], sst['edo'].isel(time = i)-sst['ryf'].isel(time = i), 
                          levels = np.arange(-.2, .21, .01), cmap = 'RdBu_r', extend = 'both')
    axs[1].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    fig.colorbar(psi, cax = axs[2], orientation = 'vertical', shrink = .5).set_label('SST [$^{\circ}$C]')
    plt.savefig(figdir+'easterlies-collaborative/figs-to-animate/sst/sst-'+str(i).zfill(3)+'.png', facecolor = 'white', bbox_inches = 'tight')
    plt.close()

In [68]:
Parallel(n_jobs = -1)(delayed(figures_parallel)(n) for n in range(len(sst['ryf']['time'])));

## SSS

In [63]:
sss = {}
sss['ryf'] = cc.querying.getvar('01deg_jra55v13_ryf9091', 'surface_salt', session, frequency = '1 daily', start_time = '2150-01-01', end_time = '2150-12-31').sel(time = slice('2150-01-01', '2150-12-31'))
sss['eup'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_up10', 'surface_salt', session, frequency = '1 daily', start_time = '2150-01-01', end_time = '2150-12-31').sel(time = slice('2150-01-01', '2150-12-31'))
sss['edo'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_down10', 'surface_salt', session, frequency = '1 daily', start_time = '2150-01-01', end_time = '2150-12-31').sel(time = slice('2150-01-01', '2150-12-31'))

In [64]:
for k in ['ryf', 'eup', 'edo']:
    sss[k] = sss[k].sel(yt_ocean = slice(None, -50))*shelf_mask

In [69]:
def figures_parallel(i):
    fig, axs = figure_along_slope()
    
    axs[0].text(0.98, 0.08, str(sss['ryf'].isel(time = i)['time.year'].values)+'-'+str(sss['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[0].transAxes);
    psi = axs[0].contourf(sss['ryf']['xt_ocean'], sss['ryf']['yt_ocean'], sss['eup'].isel(time = i)-sss['ryf'].isel(time = i), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[0].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    axs[1].text(0.98, 0.08, str(sss['ryf'].isel(time = i)['time.year'].values)+'-'+str(sss['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[1].transAxes);
    psi = axs[1].contourf(sss['ryf']['xt_ocean'], sss['ryf']['yt_ocean'], sss['edo'].isel(time = i)-sss['ryf'].isel(time = i), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[1].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    fig.colorbar(psi, cax = axs[2], orientation = 'vertical', shrink = .5).set_label('SSS')
    plt.savefig(figdir+'easterlies-collaborative/figs-to-animate/sss/sss-'+str(i).zfill(3)+'.png', facecolor = 'white', bbox_inches = 'tight')
    plt.close()

In [70]:
Parallel(n_jobs = -1)(delayed(figures_parallel)(n) for n in range(len(sss['ryf']['time'])));

## Bottom temp

In [74]:
temp = {}
temp['ryf'] = cc.querying.getvar('01deg_jra55v13_ryf9091', 'temp', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))
temp['eup'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_up10', 'temp', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))
temp['edo'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_down10', 'temp', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))

In [76]:
depth_array = temp['ryf'] * 0 + temp['ryf']['st_ocean']
max_depth = depth_array.max(dim = 'st_ocean', skipna = True)
btemp = {}
for k in ['ryf', 'eup', 'edo']:
    btemp[k] = temp[k].where(depth_array.st_ocean >= max_depth)
    btemp[k] = btemp[k].sum('st_ocean')
    btemp[k] = btemp[k]*shelf_mask

In [107]:
def figures_parallel(i):
    fig, axs = figure_along_slope()
    
    axs[0].text(0.98, 0.08, str(btemp['ryf'].isel(time = i)['time.year'].values)+'-'+str(btemp['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[0].transAxes);
    psi = axs[0].contourf(btemp['ryf']['xt_ocean'], btemp['ryf']['yt_ocean'], 
                          (btemp['eup'].isel(time = i)-btemp['ryf'].isel(time = i)).where(btemp['ryf'].isel(time = i)!= 0), 
                          levels = np.arange(-.2, .21, .01), cmap = 'RdBu_r', extend = 'both')
    axs[0].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    axs[1].text(0.98, 0.08, str(btemp['ryf'].isel(time = i)['time.year'].values)+'-'+str(btemp['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[1].transAxes);
    psi = axs[1].contourf(btemp['ryf']['xt_ocean'], btemp['ryf']['yt_ocean'], 
                          (btemp['edo'].isel(time = i)-btemp['ryf'].isel(time = i)).where(btemp['ryf'].isel(time = i)!= 0), 
                          levels = np.arange(-.2, .21, .01), cmap = 'RdBu_r', extend = 'both')
    axs[1].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    fig.colorbar(psi, cax = axs[2], orientation = 'vertical', shrink = .5).set_label('Bottom temp. [$^{\circ}$C]')
    plt.savefig(figdir+'easterlies-collaborative/figs-to-animate/btemp/btemp-'+str(i).zfill(3)+'.png', facecolor = 'white', bbox_inches = 'tight')
    plt.close()

In [108]:
Parallel(n_jobs = -1)(delayed(figures_parallel)(n) for n in range(len(btemp['ryf']['time'])));

## Bottom salt

In [95]:
salt = {}
salt['ryf'] = cc.querying.getvar('01deg_jra55v13_ryf9091', 'salt', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))
salt['eup'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_up10', 'salt', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))
salt['edo'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_down10', 'salt', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))

In [96]:
bsalt = {}
for k in ['ryf', 'eup', 'edo']:
    bsalt[k] = salt[k].where(depth_array.st_ocean >= max_depth)
    bsalt[k] = bsalt[k].sum('st_ocean')
    bsalt[k] = bsalt[k]*shelf_mask

In [97]:
def figures_parallel(i):
    fig, axs = figure_along_slope()
    
    axs[0].text(0.98, 0.08, str(bsalt['ryf'].isel(time = i)['time.year'].values)+'-'+str(bsalt['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[0].transAxes);
    psi = axs[0].contourf(bsalt['ryf']['xt_ocean'], bsalt['ryf']['yt_ocean'], 
                          (bsalt['eup'].isel(time = i)-bsalt['ryf'].isel(time = i)).where(bsalt['ryf'].isel(time = i)!= 0), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[0].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    axs[1].text(0.98, 0.08, str(bsalt['ryf'].isel(time = i)['time.year'].values)+'-'+str(bsalt['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[1].transAxes);
    psi = axs[1].contourf(bsalt['ryf']['xt_ocean'], bsalt['ryf']['yt_ocean'], 
                          (bsalt['edo'].isel(time = i)-bsalt['ryf'].isel(time = i)).where(bsalt['ryf'].isel(time = i)!= 0), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[1].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    fig.colorbar(psi, cax = axs[2], orientation = 'vertical', shrink = .5).set_label('Bottom salt.')
    plt.savefig(figdir+'easterlies-collaborative/figs-to-animate/bsalt/bsalt-'+str(i).zfill(3)+'.png', facecolor = 'white', bbox_inches = 'tight')
    plt.close()

In [98]:
Parallel(n_jobs = -1)(delayed(figures_parallel)(n) for n in range(len(bsalt['ryf']['time'])));

## Bottom age

In [100]:
age = {}
age['ryf'] = cc.querying.getvar('01deg_jra55v13_ryf9091', 'age_global', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))
age['eup'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_up10', 'age_global', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))
age['edo'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_down10', 'age_global', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2154-12-31').sel(time = slice('2150-01-01', '2154-12-31')).sel(yt_ocean = slice(None, -50))

In [101]:
bage = {}
for k in ['ryf', 'eup', 'edo']:
    bage[k] = age[k].where(depth_array.st_ocean >= max_depth)
    bage[k] = bage[k].sum('st_ocean')
    bage[k] = bage[k]*shelf_mask

In [105]:
def figures_parallel(i):
    fig, axs = figure_along_slope()
    
    axs[0].text(0.98, 0.08, str(bage['ryf'].isel(time = i)['time.year'].values)+'-'+str(bage['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[0].transAxes);
    psi = axs[0].contourf(bsalt['ryf']['xt_ocean'], bsalt['ryf']['yt_ocean'], 
                          (bage['eup'].isel(time = i)-bage['ryf'].isel(time = i)).where(bage['ryf'].isel(time = i)!= 0), 
                          levels = np.arange(-20, 21, 1), cmap = 'RdBu_r', extend = 'both')
    axs[0].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    axs[1].text(0.98, 0.08, str(bage['ryf'].isel(time = i)['time.year'].values)+'-'+str(bage['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[1].transAxes);
    psi = axs[1].contourf(bage['ryf']['xt_ocean'], bage['ryf']['yt_ocean'], 
                          (bage['edo'].isel(time = i)-bage['ryf'].isel(time = i)).where(bage['ryf'].isel(time = i)!= 0), 
                          levels = np.arange(-20, 21, 1), cmap = 'RdBu_r', extend = 'both')
    axs[1].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    fig.colorbar(psi, cax = axs[2], orientation = 'vertical', shrink = .5).set_label('Bottom age [years]')
    plt.savefig(figdir+'easterlies-collaborative/figs-to-animate/bage/bage-'+str(i).zfill(3)+'.png', facecolor = 'white', bbox_inches = 'tight')
    plt.close()

In [106]:
Parallel(n_jobs = -1)(delayed(figures_parallel)(n) for n in range(len(bage['ryf']['time'])));